Problem Statement
-----------------

You've been hired by a dog food company to try to predict why some batches of their dog food are spoiling much quicker than intended! Unfortunately this Dog Food company hasn't upgraded to the latest machinery, meaning that the amounts of the five preservative chemicals they are using can vary a lot, but which is the chemical that has the strongest effect? The dog food company first mixes up a batch of preservative that contains 4 different preservative chemicals (A,B,C,D) and then is completed with a "filler" chemical. The food scientists beleive one of the A,B,C, or D preservatives is causing the problem, but need your help to figure out which one!
Use Machine Learning with Random Forest to find out which parameter had the most predicitive power, thus finding out which chemical causes the early spoiling! So create a model and then find out how you can decide which chemical is the problem!

* Pres_A : Percentage of preservative A in the mix
* Pres_B : Percentage of preservative B in the mix
* Pres_C : Percentage of preservative C in the mix
* Pres_D : Percentage of preservative D in the mix
* Spoiled: Label indicating whether or not the dog food batch was spoiled.

In [1]:
from pyspark.sql import SparkSession

In [2]:
#Make a Spark Session
sessn = SparkSession.builder.appName('RandomForestProject').getOrCreate()

### Load csv Datatset and Show its stats

In [3]:
csv_data = sessn.read.csv(path='dog_food.csv',inferSchema=True,header=True)

In [4]:
csv_data.count()

490

In [5]:
csv_data.columns

['A', 'B', 'C', 'D', 'Spoiled']

In [6]:
csv_data.show(5)

+---+---+----+---+-------+
|  A|  B|   C|  D|Spoiled|
+---+---+----+---+-------+
|  4|  2|12.0|  3|    1.0|
|  5|  6|12.0|  7|    1.0|
|  6|  2|13.0|  6|    1.0|
|  4|  2|12.0|  1|    1.0|
|  4|  2|12.0|  3|    1.0|
+---+---+----+---+-------+
only showing top 5 rows



In [7]:
for col in csv_data.columns:
        print('The number of rows having null in feature ''',col,' '' are : ',csv_data.where(csv_data[col].isNull()).count())

The number of rows having null in feature  A   are :  0
The number of rows having null in feature  B   are :  0
The number of rows having null in feature  C   are :  0
The number of rows having null in feature  D   are :  0
The number of rows having null in feature  Spoiled   are :  0


In [8]:
csv_data.describe().show()

+-------+------------------+------------------+------------------+------------------+-------------------+
|summary|                 A|                 B|                 C|                 D|            Spoiled|
+-------+------------------+------------------+------------------+------------------+-------------------+
|  count|               490|               490|               490|               490|                490|
|   mean|  5.53469387755102| 5.504081632653061| 9.126530612244897| 5.579591836734694| 0.2857142857142857|
| stddev|2.9515204234399057|2.8537966089662063|2.0555451971054275|2.8548369309982857|0.45221563164613465|
|    min|                 1|                 1|               5.0|                 1|                0.0|
|    max|                10|                10|              14.0|                10|                1.0|
+-------+------------------+------------------+------------------+------------------+-------------------+



In [11]:
csv_data.printSchema()

root
 |-- A: integer (nullable = true)
 |-- B: integer (nullable = true)
 |-- C: double (nullable = true)
 |-- D: integer (nullable = true)
 |-- Spoiled: double (nullable = true)



##### It is evident from above that there are no observations with null values

### Convert csv_data into Vector form to be accepted by ML Algos

In [9]:
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler

In [12]:
assembler = VectorAssembler(inputCols=['A', 'B', 'C', 'D'],outputCol = 'features')

In [13]:
vec_data = assembler.transform(csv_data)

In [14]:
vec_data.show(5)

+---+---+----+---+-------+------------------+
|  A|  B|   C|  D|Spoiled|          features|
+---+---+----+---+-------+------------------+
|  4|  2|12.0|  3|    1.0|[4.0,2.0,12.0,3.0]|
|  5|  6|12.0|  7|    1.0|[5.0,6.0,12.0,7.0]|
|  6|  2|13.0|  6|    1.0|[6.0,2.0,13.0,6.0]|
|  4|  2|12.0|  1|    1.0|[4.0,2.0,12.0,1.0]|
|  4|  2|12.0|  3|    1.0|[4.0,2.0,12.0,3.0]|
+---+---+----+---+-------+------------------+
only showing top 5 rows



In [17]:
final_data = vec_data['features','Spoiled']

### Build Random Forest Model

In [15]:
from pyspark.ml.classification import RandomForestClassifier

In [20]:
RFC_ob = RandomForestClassifier(featuresCol='features',labelCol='Spoiled',numTrees=100)

In [21]:
RFCModel = RFC_ob.fit(final_data)

In [22]:
RFCModel.featureImportances

SparseVector(4, {0: 0.0192, 1: 0.0228, 2: 0.9339, 3: 0.0241})

###### From above Feature Importance Stats, it looks like Preservative C is causing the Problem